# Test 3DVar and 4DVar implementation

In [1]:
import torch
from deepda import apply_3DVar, apply_4DVar, forwardModel_r
from math import ceil

device = "cpu"

In [2]:
def H(x: torch.Tensor):
    return x

In [3]:
B = torch.eye(3, device=device)
R = torch.eye(3, device=device)
y = torch.tensor([10., 20., 30.], device=device)
xb = torch.zeros_like(y, device=device)

In [4]:
apply_3DVar(H, B, R, xb, y, learning_rate=2)

Iterations: 0, J: 1400.0, Norm of J gradient: 74.83314514160156
Iterations: 1, J: 1184.0, Norm of J gradient: 62.22539520263672
Iterations: 2, J: 1017.4765014648438, Norm of J gradient: 50.39654541015625
Iterations: 3, J: 898.0814208984375, Norm of J gradient: 39.807682037353516
Iterations: 4, J: 818.659912109375, Norm of J gradient: 30.810375213623047
Iterations: 5, J: 768.9251708984375, Norm of J gradient: 23.481937408447266
Iterations: 6, J: 740.5263671875, Norm of J gradient: 18.005840301513672
Iterations: 7, J: 728.07568359375, Norm of J gradient: 14.986849784851074
Iterations: 8, J: 727.2283935546875, Norm of J gradient: 14.758960723876953
Iterations: 9, J: 733.88916015625, Norm of J gradient: 16.46552276611328
Iterations: 10, J: 744.373046875, Norm of J gradient: 18.841018676757812
Iterations: 11, J: 755.5263061523438, Norm of J gradient: 21.076297760009766
Iterations: 12, J: 764.7009887695312, Norm of J gradient: 22.75099754333496
Iterations: 13, J: 769.8867797851562, Norm of J

tensor([ 5.0000, 10.0000, 15.0000])

In [5]:
# We define the control parameters here
rayleigh = 35
prandtl = 10.
b = 8./3.
# rayleigh = 0.
# prandtl = 0.
# b = 0.
# initial condition for the true reference trajectory
x0 = torch.tensor([0., 1., 2.], device=device)

# integration time parameter
dt = 1.e-3      # This is time step size
T = 2.         # Total integration time, can be as short as 10 to speed things up
n_steps = ceil(T / dt)
time = torch.linspace(0., T, n_steps + 1, device=device)  # array of discrete times

# numerical integration given initial conditions and control parameters
xt = forwardModel_r(x0, time, rayleigh, prandtl, b)

In [6]:
sigobs = 2.  # standard deviation of the observation noise
# How often do we observe the true state?
dtobs = 0.5  # time between observations
nobs = ceil(T / dtobs)  # number of times observations are performed
# no observation at t=0
gap = int(dtobs / dt)  # number of time steps between each observation
time_obs = time[gap::gap]
# Generate vector of observations
y = torch.zeros((x0.size(0), nobs), device=device)
R = 1e-8 * torch.diag(torch.tile(torch.tensor(sigobs**2, device=device), (x0.size(0),)))
sqrt_s = torch.sqrt(R)
# y = Hxt
y = H(xt[:, gap::gap])
# compute observation error
noise = sqrt_s @ torch.randn(size=y.shape, device=device)
# y = Hxt + epsilon
y = y + noise
y = y.T

In [7]:
xb = torch.randn((3,), device=device)
apply_4DVar(nobs, time_obs, gap, forwardModel_r, H, B, R, xb, y, model_args=(rayleigh, prandtl, b), learning_rate=2.5e-3, max_iterations=1000)

Iterations: 0, J: 4282035200.0, Norm of J gradient: 2898978304.0
Iterations: 1, J: 4270585344.0, Norm of J gradient: 2906381312.0
Iterations: 2, J: 4259095552.0, Norm of J gradient: 2913833728.0
Iterations: 3, J: 4247599104.0, Norm of J gradient: 2921376512.0
Iterations: 4, J: 4236096000.0, Norm of J gradient: 2928928768.0
Iterations: 5, J: 4224557568.0, Norm of J gradient: 2936536832.0
Iterations: 6, J: 4212941312.0, Norm of J gradient: 2944211712.0
Iterations: 7, J: 4201354496.0, Norm of J gradient: 2951879168.0
Iterations: 8, J: 4189708544.0, Norm of J gradient: 2959657728.0
Iterations: 9, J: 4178027776.0, Norm of J gradient: 2967427072.0
Iterations: 10, J: 4166319616.0, Norm of J gradient: 2975260672.0
Iterations: 11, J: 4154590464.0, Norm of J gradient: 2983183360.0
Iterations: 12, J: 4142810112.0, Norm of J gradient: 2991124736.0
Iterations: 13, J: 4131024128.0, Norm of J gradient: 2999118080.0
Iterations: 14, J: 4119175680.0, Norm of J gradient: 3007200000.0
Iterations: 15, J: 4

tensor([0.6131, 0.0389, 0.7731])